## Import lib

In [1]:
import os
import pandas as pd
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

from sklearn import preprocessing
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, roc_auc_score, recall_score, precision_score, confusion_matrix,accuracy_score,classification_report
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
# from sklearn.datasets import load_files
# sys.path.append('..') 
# import helpers.data_mining_helpers as dmh

# import nltk
# nltk.download('punkt')
# import seaborn as sns
# from sklearn.decomposition import PCA
# from sklearn import preprocessing, metrics, decomposition, pipeline, dummy
# from sklearn.preprocessing import binarize
# from sklearn.metrics.pairwise import cosine_similarity



## Data input

In [11]:
# train_file_path = '..\\training.csv'
# test_file_path = '..\\public_processed.csv'
# test_file_answer_path = '..\\public.csv'
# test_private_file_path = '..\\private_1_processed.csv'

train_file_path = 'training.csv'
test_file_path = 'public_processed.csv'
test_file_answer_path = 'public.csv'
test_private_file_path = 'private_1_processed.csv'

df = pd.read_csv(train_file_path)
testing_df = pd.read_csv(test_file_path)
testing_answer_df = pd.read_csv(test_file_answer_path)
testing_private_df = pd.read_csv(test_private_file_path)

df.shape : (8688526, 26)  |||  test_df.shape : (600182, 25)

In [12]:
# training_df = df.copy()
# test_df = testing_df.copy()
training_df = df
test_df = testing_df
y = training_df['label']
training_df = training_df.drop(['label','txkey'], axis=1)
test_df = test_df.drop(['txkey'], axis=1)

In [13]:
train_num = training_df.shape[0]
test_num = test_df.shape[0]
allData = pd.concat([training_df ,test_df])


In [15]:

# print(df.shape)
# print(test_df.shape)

# Rename the columns to match the previous code
# df.columns = ['txkey', 'locdt', 'loctm', 'chid', 'cano', 'locdt']
# test_df.columns = ['sentence', 'score', 'provider']

# Combine the two dataframes
# X = test_df
# display(test_df.head())
# display(df.head())
# Randomly select 3000 rows and drop duplicates and missing values
# X = X.sample(n=3000, random_state=42, replace=False)
# X.dropna(inplace=True)
# X.drop_duplicates(inplace=True)
# display(X)


label_encoder_chid = LabelEncoder()
label_encoder_cano = LabelEncoder()
label_encoder_mchno = LabelEncoder()
label_encoder_acqic = LabelEncoder()
label_encoder_etymd = LabelEncoder()
label_encoder_mcc = LabelEncoder()
label_encoder_stocn = LabelEncoder()
label_encoder_scity = LabelEncoder()
label_encoder_stscd = LabelEncoder()
label_encoder_hcefg = LabelEncoder()
label_encoder_csmcu = LabelEncoder()

allData['chid'] = label_encoder_chid.fit_transform(allData['chid'])
allData['cano'] = label_encoder_cano.fit_transform(allData['cano'])
allData['mchno'] = label_encoder_mchno.fit_transform(allData['mchno'])
allData['acqic'] = label_encoder_acqic.fit_transform(allData['acqic'])
allData['etymd']= label_encoder_etymd.fit_transform(allData['etymd'])
allData['mcc']= label_encoder_mcc.fit_transform(allData['mcc'])
allData['stocn']= label_encoder_stocn.fit_transform(allData['stocn'])
allData['scity']= label_encoder_scity.fit_transform(allData['scity'])
allData['stscd']= label_encoder_stscd.fit_transform(allData['stscd'])
allData['hcefg']= label_encoder_hcefg.fit_transform(allData['hcefg'])
allData['csmcu']= label_encoder_csmcu.fit_transform(allData['csmcu'])


allData['chid'].astype("category")
allData['cano'].astype("category")
allData['mchno'].astype("category")
allData['acqic'].astype("category")
allData['etymd'].astype("category")
allData['mcc'].astype("category")
allData['stocn'].astype("category")
allData['scity'].astype("category")
allData['stscd'].astype("category")
allData['hcefg'].astype("category")
allData['csmcu'].astype("category")

allData['contp'].astype("category")
allData['etymd'].astype("category")
allData['ecfg'].astype("category")
allData['insfg'].astype("category")
allData['bnsfg'].astype("category")
allData['ovrlt'].astype("category")
allData['flbmk'].astype("category")
allData['flg_3dsmk'].astype("category")

# ##test
# test_df['chid'] = label_encoder.fit_transform(test_df['chid'])
# test_df['cano'] = label_encoder.fit_transform(test_df['cano'])
# test_df['mchno'] = label_encoder.fit_transform(test_df['mchno'])
# test_df['acqic'] = label_encoder.fit_transform(test_df['acqic'])
# test_df['etymd']= label_encoder.fit_transform(test_df['etymd'])
# test_df['mcc']= label_encoder.fit_transform(test_df['mcc'])
# test_df['stocn']= label_encoder.fit_transform(test_df['stocn'])
# test_df['scity']= label_encoder.fit_transform(test_df['scity'])
# test_df['stscd']= label_encoder.fit_transform(test_df['stscd'])
# test_df['hcefg']= label_encoder.fit_transform(test_df['hcefg'])
# test_df['csmcu']= label_encoder.fit_transform(test_df['csmcu'])


0         0
1         0
2         0
3         0
4         0
         ..
600177    0
600178    0
600179    0
600180    0
600181    0
Name: flg_3dsmk, Length: 9288708, dtype: category
Categories (2, int64): [0, 1]

In [16]:
training_df = allData[:train_num]
test_df = allData[train_num:]

In [10]:
training_df[:10]

,locdt,loctm,chid,cano,contp,etymd,mchno,acqic,mcc,conam,...,flam1,stocn,scity,stscd,ovrlt,flbmk,hcefg,csmcu,csmam,flg_3dsmk
0,46,101812,84d2dc85d4da6a7fa284a11a4290d7e9a969163dcb4d82...,3dd5bf1e29e5e0baa789ce692fe5dbd34ff05173acf351...,5,1.0,cad752c5d05d2bdcc30d64fa4e68404c2d1f7be5d14d52...,8f6b3ff512a001e0d1988c6cd888ef8c74112fb71117e5...,375.0,1524.0,...,1529,0.0,15759.0,NaN,0,0,6.0,70.0,1529,0
1,17,85509,9a8cf5d0afd729cb7876f6b3172152c7c9c6fabd40515c...,8cb13f9b38c7bbc02d210e580dcbbcbb6c95bf18bc3320...,5,8.0,4356c6642ef6e837543b577c7ee3ffa92b4b8fcfb57254...,379166ff4a62dac343b4b734188aa618716cc496e48b65...,282.0,101.0,...,101,0.0,15759.0,NaN,0,0,8.0,70.0,101,0
2,44,212641,dcc1389a5765d6f53152cf85970fbe78a83fd3d1c299b0...,1ec32868e5e1d5ff8df56737c2a91326cbfe3364382de6...,5,4.0,5b7eff061f8896aac4339ea35c25f8bb956a43bc486460...,8f6b3ff512a001e0d1988c6cd888ef8c74112fb71117e5...,288.0,117.0,...,116,0.0,15759.0,NaN,0,0,6.0,70.0,116,0
3,42,102702,577f2329d1eccd59ba0abaf6113bb78dcd575badcbc57f...,4359dca1ac6a835eceb2bc0dd6b0b710f030c3499126e9...,5,1.0,cad752c5d05d2bdcc30d64fa4e68404c2d1f7be5d14d52...,36684976be1f529e6e2a32c9edab4cf8e364b2b916ae2c...,375.0,1676.0,...,1683,0.0,15759.0,NaN,0,0,6.0,70.0,1683,0
4,31,185737,fff6b4126c40620b1fbb11d4de02cd67b9e95071caa40b...,a3837f2905383f235a72679482c5f02e40f2a8ca29750d...,5,5.0,50d5b02ce3fc88723438c2a29cfdb04be4a1a11280ddb6...,379166ff4a62dac343b4b734188aa618716cc496e48b65...,406.0,14.0,...,14,0.0,15759.0,NaN,0,0,6.0,70.0,14,0
5,53,154344,7bccecefb9c0271fec1b07dcce8b64b8f24a8d45e16376...,84b40264cc97d2ce1014a4ad5ca15da3fd85cb0aaa0aeb...,5,4.0,9447d4c28619ff5cba6dd79ab863fa6f365c1624fcf183...,13624921218432873e0cfa1966e41376844b387695a0ee...,335.0,2780.0,...,2789,0.0,17014.0,NaN,0,0,6.0,70.0,2789,0
6,53,141651,ba6cbd06d4d43b3b6170c9bc29d0ab01d675db5b8b16d9...,ceecd9e6204f6105e2d47cbb6c40359ec7eefaf743bf6c...,5,8.0,de8ce15e75639cac3466907561cc112fa5e9519fff4582...,bcf6370b700fb26d1fab05dad5de442b2bf37689cb55fd...,309.0,138.0,...,138,0.0,15759.0,NaN,0,0,6.0,70.0,138,1
7,31,102343,b30d2725ca2bc993f351edeb8a89fdf4df597ad8ca23ae...,da18d6d5d3b4d9eafc40bc71468077b1188422ddb394ae...,5,5.0,f266cbc5720f6f0114813a3bbabc61c07989ac24845e97...,8f6b3ff512a001e0d1988c6cd888ef8c74112fb71117e5...,320.0,30.0,...,30,0.0,16115.0,NaN,0,0,6.0,70.0,30,0
8,21,232815,71d423387298bd70d8b7c790b533ab35ffaaba589a4d36...,a2da56ef94c2fd1a845637a232ff6106a61e3e8df0f144...,5,8.0,4ef6971e3a32b67a04e1bb10d981306faf3c422e5cb467...,bcf6370b700fb26d1fab05dad5de442b2bf37689cb55fd...,282.0,680.0,...,681,0.0,15759.0,NaN,0,0,6.0,70.0,681,1
9,31,113900,d98d0c5842c8c4d83c00fc04d20128e759760a81e22c1f...,5f8ca305864006b3db759363e944d01a8e7b0dc4166835...,5,8.0,dd280b4f35a02701142ba7de47abb9736deca3eed86474...,46f63555d29f177290dfe20e989146fb033e453f1d5ff3...,320.0,250.0,...,250,0.0,13451.0,NaN,0,0,6.0,70.0,250,0


In [9]:
display(test_df['csmcu'].head())
display(testing_df['csmcu'].head())

0    70.0
1    70.0
2     NaN
3    70.0
4    68.0
Name: csmcu, dtype: float64

0    70.0
1    70.0
2     NaN
3    70.0
4    68.0
Name: csmcu, dtype: float64

## Data preprocess.

type:  
type0 : txkeys     
type1 (times) : locdt loctm        
type2 (money) : conam flam1 ovrlt csmcu csmam      
type3 (person) : chid cano     
type4 (places) : mchno stocn scity     
type5 (other trade types) : contp etymd acqic mcc ecfg insfg iterm bnsfg stscd flbmk hcefg flg_3dsmk       


In [5]:




fraud_df = training_df[df['label'] == 1]
non_fraud_df = training_df[df['label'] == 0]
# print(fraud_df.shape)
# print(non_fraud_df.shape)
# print(training_df.shape)


In [6]:
# each feature should use ifts own label encoder so that the unseen value can be handled
# try:
#     print(label_encoder.transform([-1]))
# except ValueError:
#     print("unseen value")


In [7]:
# training_df.drop(['txkey'], axis=1, inplace=True)

In [ ]:
# print(fraud_df[col].shape[0]/ training_df[col].shape[0])
# print(non_fraud_df[col].shape[0]/ training_df[col].shape[0])
for col in training_df.columns:  
    print(f"col: {col}")
    # print(f"nunique: {training_df[col].nunique()}")
    # print(f"unique values: {training_df[col].unique()}")
    # print(f"fraud nunique: {fraud_df[col].nunique()}")
    # print(f"fraud unique values: {fraud_df[col].unique()}")
    # print(f"non fraud nunique: {non_fraud_df[col].nunique()}")
    # print(f"non fraud unique values: {non_fraud_df[col].unique()}")
    # print(fraud_df[col].nunique()/ training_df[col].nunique())
    # print(non_fraud_df[col].nunique()/ training_df[col].nunique()) 
    unique_values = fraud_df[col].unique()
    # print(f"unique values: {unique_values}")

    #average_ratio
    # average_ratio = 0
    # for value in unique_values:
    #     average_ratio += fraud_df[col][fraud_df[col] == value].shape[0]/ training_df[col][training_df[col] == value].shape[0]
    # average_ratio /= len(unique_values)
    # print(f"average_ratio: {average_ratio}")

    #real_ratio
    # unique_values = fraud_df[col].unique()
    # numerator = np.sum(np.isin(fraud_df[col], unique_values))
    # denominator = np.sum(np.isin(training_df[col], unique_values))
    # average_ratio = numerator/denominator
    # print(f"real_ratio: {average_ratio}")

    #max and min ratio top 10 and plot
    unique_values = fraud_df[col].unique()
    ratio_list = []
    for value in unique_values:
        ratio = fraud_df[col][fraud_df[col] == value].shape[0]/ training_df[col][training_df[col] == value].shape[0]
        ratio_list.append(ratio)
    ratio_list.sort(reverse=True)
    if len(ratio_list) > 10:
        print(f"max ratio top 10: {ratio_list[:10]}")
        plt.figure(figsize=(10, 5))
        plt.bar(np.arange(1, 11), ratio_list[:10])
        plt.title(f"Max Ratio of Fraud Transactions for {col}")
        plt.xlabel("Unique Values")
        plt.ylabel("Max Ratio of Fraud Transactions")
        plt.show()
        print(f"min ratio top 10: {ratio_list[-10:]}")
        plt.figure(figsize=(10, 5))
        plt.bar(np.arange(1, 11), ratio_list[-10:])
        plt.title(f"Min Ratio of Fraud Transactions for {col}")
        plt.xlabel("Unique Values")
        plt.ylabel("Min Ratio of Fraud Transactions")
        plt.show()
    else:
        print(f"max ratio: {ratio_list}")
        plt.figure(figsize=(10, 5))
        plt.bar(unique_values, ratio_list)
        plt.title(f"Max Ratio of Fraud Transactions for {col}")
        plt.xlabel("Unique Values")
        plt.ylabel("Max Ratio of Fraud Transactions")
        plt.show()
    print('\n')


Average ratio:  all unique value in fraud_df and average (amount in fraud_df / amount in training_df) for each feature(column)   
col: label      

---
col: cano       
col: chid       
col: scity      
col: mchno      
col: conam      

---
col: acqic      
col: stocn      
col: csmcu       
col: stscd      
col: flam1      
col: hcefg      
col: mcc        

---
col: csmam      
col: loctm      

---
col: locdt      
col: contp      
col: etymd      
col: ecfg       
col: insfg     
col: iterm      
col: bnsfg     
col: ovrlt   
col: flbmk   
col: flg_3dsmk 

In [17]:
# y = training_df['label']
# X = training_df.drop(['label','txkey'], axis=1)
X = training_df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## training & testing.
### LightGBM

In [18]:



# df_sample = df.sample(n=300000, random_state=42)
# train_df, val_df = train_test_split(df_sample, test_size=0.2, random_state=42)

# # Define the features and target variable
# #featues 前面preprocess先做好這邊在註解掉
# features = [ 'locdt','loctm','contp','etymd','mcc','conam','ecfg','insfg','iterm','bnsfg','flam1','stocn','scity','ovrlt','flbmk','hcefg','csmcu','csmam','flg_3dsmk'] 
# target = 'label'

# # Split the training and validation datasets into features and target variable
# X_train = train_df[features]
# y_train = train_df[target]
# X_val = val_df[features]
# y_val = val_df[target]

# # Train the model using LightGBM

# # params = {
# #     'objective': 'binary',
# #     'metric': 'binary_logloss',
# #     'boosting_type': 'dart',
# #     'num_leaves': 50,  
# #     'learning_rate': 0.01,  
# #     'feature_fraction': 0.8  
# # }
# params = {'objective': 'binary',
#           'boosting_type': 'gbdt',
#           'metric': 'auc',
#           'learning_rate': 0.007,
#           'num_leaves': 2**8,
#           'max_depth': -1,
#           'tree_learner':'serial',
#           'colsample_bytree': 0.5,
#           'subsample_freq':1,
#           'subsample':0.7,
#           'n_estimators':10000,
#           'min_data_in_leaf': 106,
#           'max_bin':255,
#           'verbosity': -1,
#           'early_stopping_rounds':100,
#          }


# train_data = lgb.Dataset(X_train, label=y_train)
# val_data = lgb.Dataset(X_val, label=y_val)

# model = lgb.train(params, train_data, num_boost_round=10000, valid_sets=[train_data, val_data])
# model = lgb.train(params, train_data, num_boost_round=1000, valid_sets=[train_data, val_data], early_stopping_rounds=50)

categories = ['chid', 'cano', 'contp', 'etymd', 'mchno', 'acqic',
              'mcc',  'ecfg', 'insfg',  'bnsfg', 'stocn', 'scity',
              'stscd', 'ovrlt', 'flbmk', 'hcefg', 'csmcu', 'flg_3dsmk']

params = {
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 60,
    'max_depth': 6,
    'learning_rate': 0.005,
    'n_estimators': 3000,
    'boosting_type': 'gbdt',
}   

train_data = lgb.Dataset(X_train, label=y_train,
                         categorical_feature=categories)
test_data = lgb.Dataset(X_test, label=y_test,
                        reference=train_data)

bst = lgb.train(params, train_data,num_boost_round = 100,  valid_sets=[test_data],categorical_feature=categories)

/home/swc/DM_Final/lib/python3.8/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 25694, number of negative: 6925126
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.216511 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 39625
[LightGBM] [Info] Number of data points in the train set: 6950820, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.003697 -> initscore=-5.596654
[LightGBM] [Info] Start training from score -5.596654
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

validation.

In [27]:
y_pred_prob = bst.predict(X_test, num_iteration=bst.best_iteration)
y_pred2 = [1 if x >= 0.5 else 0 for x in y_pred_prob]  # Assuming binary classification, adjust as needed
# y_pred = model.predict(X_val)
# print(y_pred)
# # Get the true labels from the validation set
# y_true = val_data.label

# # Calculate F1 score
# f1 = f1_score(y_val, y_pred)
# print(f'F1 Score on Validation Set: {f1}')

# y_pred = pd.Series(bst.predict(X_test, num_iteration=bst.best_iteration)>0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred2)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred2,digits=5))

Accuracy: 0.9977740768576503
Classification Report:
              precision    recall  f1-score   support

           0    0.99811   0.99966   0.99888   1731371
           1    0.83785   0.48287   0.61266      6335

    accuracy                        0.99777   1737706
   macro avg    0.91798   0.74127   0.80577   1737706
weighted avg    0.99753   0.99777   0.99748   1737706



### XGboost

In [38]:
from xgboost import XGBClassifier
xgboostModel = XGBClassifier( n_estimators=12000, learning_rate= 0.005,enable_categorical=True,
                             objective = "binary:hinge",eval_metric="auc",max_depth=12)#grow_policy="lossguide"
xgboostModel.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.005, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=12, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=12000, n_jobs=None,
              num_parallel_tree=None, objective='binary:hinge', ...)

In [39]:
y_pred = xgboostModel.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred,digits=5))

Accuracy: 0.9986712366763998
Classification Report:
              precision    recall  f1-score   support

           0    0.99890   0.99977   0.99933   1731371
           1    0.91677   0.69897   0.79319      6335

    accuracy                        0.99867   1737706
   macro avg    0.95783   0.84937   0.89626   1737706
weighted avg    0.99860   0.99867   0.99858   1737706



## predict

Use pubic data to vlidation the score.

Predict part.

In [29]:
# display(training_df.head())
# display(X_test.head())
# display(test_df.head())
# predict_data = test_df.drop(['txkey'], axis=1)
predict_data = test_df
y_pred = pd.Series(bst.predict(predict_data, num_iteration=bst.best_iteration)>0.5).astype(int)
# pridict = pd.DataFrame(test_df['txkey'] )
# pridict['pred'] = y_pred # put here
# display(pridict)

In [31]:
# test_df = pd.read_csv(test_file_path)
pridict = pd.DataFrame(testing_df['txkey'] )
pridict['pred'] = y_pred 
# display(y_pred )
# print(np.nonzero(y_pred))
# print(y_pred[1771])
# display(pridict.iloc[1900])
display(pridict)

,txkey,pred
0,a2c1209018e4e52e04f6fabb48f05f1b8bc09dc838ff6c...,0
1,16c4880500059e01553789be11bbb50753b7acaae7b95b...,0
2,623c56be3bee87724e3d119c271d9ed098eeda84233183...,0
3,250da12187059cf6e3a3066656a2919d08ceb8207efd55...,0
4,4b268e0da036f44cbbb056ddfac6a28ea336d9cf299843...,0
...,...,...
600177,741a391b2c2114491d1d18acb1a892341b9280e1e3529f...,0
600178,76f613595631fb35938bf10f5b7abe3f60a3d34ccf5f44...,0
600179,c32e846a4469e4869e2df8f8e160fba4e829b6beebb44d...,0
600180,d641cd25a830de759cd55b4a643d71eadb770bc917aed3...,0


## output

In [32]:
pridict.to_csv('output.csv', index=False)